In [39]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [40]:
import os
import numpy as np
from lxml import etree

# Define the path to the annotations directory
ann_dir = "Annotations"

# Define the list of class names
class_names = ["Hydrant", "Mail Box & Street Names", "Traffic Light","Human Beings"]

# Initialize the list of bounding boxes and labels
bboxes_list = []
labels_list = []

# Loop over the annotated XML files
for ann_file in os.listdir(ann_dir):
    # Parse the XML file
    root = etree.parse(os.path.join(ann_dir, ann_file))

    # Extract the image width and height
    width = int(root.find("size/width").text)
    height = int(root.find("size/height").text)

    # Loop over the objects in the XML file
    for obj in root.findall("object"):
        # Extract the class name
        class_name = obj.find("name").text

        # Extract the bounding box coordinates
        bbox = obj.find("bndbox")
        xmin = int(float(bbox.find("xmin").text))
        ymin = int(float(bbox.find("ymin").text))
        xmax = int(float(bbox.find("xmax").text))
        ymax = int(float(bbox.find("ymax").text))

        # Convert the bounding box coordinates to relative values
        xmin = xmin / width
        ymin = ymin / height
        xmax = xmax / width
        ymax = ymax / height

        # Append the bounding box and label to the lists
        bboxes_list.append([xmin, ymin, xmax, ymax])
        labels_list.append(class_names.index(class_name))

# Convert the lists to NumPy arrays
bboxes = np.array(bboxes_list)
labels = np.array(labels_list)


In [30]:
# Define the input shape for the model
input_shape = (None, None, 3)

# Define the input layer
inputs = layers.Input(shape=input_shape)

# Define the backbone network
backbone = keras.applications.ResNet50V2(
    include_top=False, weights="imagenet", input_tensor=inputs
)

# Define the Region Proposal Network (RPN)
rpn = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(backbone.output)

rpn_class = layers.Conv2D(2, (1, 1), activation="sigmoid")(rpn)

rpn_bbox = layers.Conv2D(4, (1, 1))


In [41]:
from tensorflow.keras.models import Sequential

In [42]:
def faster_rcnn(num_classes):

    # Backbone CNN
    backbone = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(None, None, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2))
    ])

    # Region Proposal Network (RPN)
    rpn = models.Sequential([
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(2 * 9, (1, 1), activation='sigmoid'),
        layers.Lambda(lambda x: regions.RegionProposalLayer()([x[0], x[1]]))
    ])

    # Fast R-CNN detector
    fast_rcnn_model = models.Sequential([
        layers.RoiPooling2D(7, 7),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(4 * num_classes, activation='linear'),
        layers.Lambda(lambda x: regions.DetectionsLayer(num_classes)([x[0], x[1]]))
    ])

    # Inputs
    backbone_input = layers.Input(shape=(None, None, 3))
    rpn_input = layers.Input(shape=(None, None, 256))
    roi_input = layers.Input(shape=(None, 4))

    # Intermediate layers
    backbone_output = backbone(backbone_input)
    rpn_class_output, rpn_bbox_output = rpn(backbone_output)
    anchors = layers.AnchorBoxes()(backbone_output)
    proposals = layers.ProposalLayer()([rpn_class_output, rpn_bbox_output, anchors])
    detections_class_output, detections_bbox_output = fast_rcnn_model([backbone_input, proposals])

    # Final model
    model = models.Model(inputs=[backbone_input, rpn_input, roi_input], 
                         outputs=[rpn_class_output, rpn_bbox_output, detections_class_output, detections_bbox_output],
                         name='faster_rcnn')

    return model


In [43]:

model = faster_rcnn(10) # assuming we have 10 object classes

optimizer = keras.optimizers.Adam(lr=0.001)

# Define the loss function for the RPN
rpn_cls_loss_fn = regions.rpn_cls_loss(num_classes)
rpn_reg_loss_fn = regions.rpn_reg_loss()
rpn_loss_fn = lambda y_true, y_pred: rpn_cls_loss_fn(y_true[:,:,:,-18:-9], y_pred[0]) + rpn_reg_loss_fn(y_true[:,:,:,-9:], y_pred[1])

model.compile(
    optimizer=optimizer,
    loss=[rpn_loss_fn, keras.losses.MeanSquaredError()],
)


# Define the data generator for the model
def data_generator(batch_size=32):
    while True:
        # Randomly select images and their corresponding bounding boxes and labels
        indices = np.random.choice(len(images), size=batch_size, replace=False)
        batch_images = [images[i] for i in indices]
        batch_bboxes = [bboxes[i] for i in indices]
        batch_labels = [labels[i] for i in indices]

        # Preprocess the images and their corresponding bounding boxes and labels
        batch_images, batch_bboxes = preprocess_data(
            batch_images, batch_bboxes, input_shape
        )

        # Generate the anchor boxes for the images
        anchor_boxes = generate_anchor_boxes(input_shape)

        # Generate the ground truth labels and bounding boxes for the anchor boxes
        rpn_labels, rpn_bbox_targets = generate_rpn_targets(
            batch_bboxes, anchor_boxes, batch_labels
        )

        # Yield the data
        yield (
            batch_images,
            (rpn_labels, rpn_bbox_targets),
        )

# Train the model
model.fit(
    data_generator(),
    steps_per_epoch=1000,
    epochs=10,
)



AttributeError: module 'django.db.models' has no attribute 'Sequential'